Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization](https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether a document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. [CLS] token is inserted at the beginning of each sentence, so is [SEP] token at the end. Interval segment embedding and positional embedding are added upon the token embedding as the input of the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as the input for the summarization model. The summarization layer predicts the probability for each  sentence being included in the summary. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~2 hours to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 10 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 


In [1]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
## Set USE_PREPROCSSED_DATA = True to skip the data preprocessing
USE_PREPROCSSED_DATA = True

### Configuration


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

import pandas as pd
import scrapbook as sb

/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu


### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [5]:
pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

,model_name
0,bert-base-uncased
1,distilbert-base-uncased


In [6]:
# Transformer model being used
MODEL_NAME = "distilbert-base-uncased"

Also, we need to install the dependencies for pyrouge.

# dependencies for ROUGE-1.5.5.pl
Run the following commands in your terminal to install XML parsing C library.

1. sudo apt-get update
1. sudo apt-get install expat
1. sudo apt-get install libexpat-dev -y

Run the following commands in your terminal to install other pre-requistes for using pyrouge.
1. sudo cpan install XML::Parser
1. sudo cpan install XML::Parser::PerlSAX
1. sudo cpan install XML::DOM

Download ROUGE-1.5.5 from https://github.com/andersjo/pyrouge/tree/master/tools/ROUGE-1.5.5.
Run the following command in your terminal.
* pyrouge_set_rouge_path $ABSOLUTE_DIRECTORY_TO_ROUGE-1.5.5.pl

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle downloading. Functions defined specific in [cnndm.py](../../utils_nlp/dataset/cnndm.py) are unique to CNN/DM dataset that's preprocessed by harvardnlp. However, it provides a skeleton of how to preprocessing text into the format that model preprocessor takes: sentence tokenization and work tokenization. 

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that model finetuning needed. Assuming you have (1) all articles and (2) target summaries, each in a file and line-breaker separated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The algorithms for the sentence selection are "greedy" and "combination" and can be found in [sentence_selection.py](../../utils_nlp/dataset/sentence_selection.py)
3. convert each example to  the desired format for extractive summarization
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the total number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization or Byte Pair Encoding (BPE) subword tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the transformer tokenizer's vocabulary.
    - segment ids are generated and added
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the transformer model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data preprocessing, here we use NLTK for data preprocessing. 

##### [Option 1] Preprocess  data (Please skil this part if you choose to use preprocessed data)
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [7]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
CHUNK_SIZE=200
if not QUICK_RUN:
    TOP_N = -1
    CHUNK_SIZE = 2000

In [19]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH)

I0103 05:29:37.485339 140060135520064 utils.py:173] Opening tar file /tmp/tmpjd6tv6g9/cnndm.tar.gz.
I0103 05:29:37.487093 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/test.txt.src already extracted.
I0103 05:29:37.777695 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/test.txt.tgt.tagged already extracted.
I0103 05:29:37.804513 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/train.txt.src already extracted.
I0103 05:29:45.345131 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/train.txt.tgt.tagged already extracted.
I0103 05:29:45.963999 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/val.txt.src already extracted.
I0103 05:29:46.300785 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/val.txt.tgt.tagged already extracted.


Preprocess the data and save the data to disk.

In [20]:
processor = ExtSumProcessor(model_name=MODEL_NAME)
ext_sum_train = processor.preprocess(train_dataset, train_dataset.get_target(), oracle_mode="greedy")
ext_sum_test = processor.preprocess(test_dataset, test_dataset.get_target(),oracle_mode="greedy")

I0103 05:29:49.643863 140060135520064 tokenization_utils.py:379] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [21]:
save_path = os.path.join(DATA_PATH, "processed")
train_files = ExtSumProcessedData.save_data(
    ext_sum_train, is_test=False, save_path=save_path, chunk_size=CHUNK_SIZE
)
test_files = ExtSumProcessedData.save_data(
    ext_sum_test, is_test=True, save_path=save_path, chunk_size=CHUNK_SIZE
)

In [22]:
train_files

['/tmp/tmpjd6tv6g9/processed/0_train',
 '/tmp/tmpjd6tv6g9/processed/1_train',
 '/tmp/tmpjd6tv6g9/processed/2_train',
 '/tmp/tmpjd6tv6g9/processed/3_train',
 '/tmp/tmpjd6tv6g9/processed/4_train']

In [23]:
test_files

['/tmp/tmpjd6tv6g9/processed/0_test',
 '/tmp/tmpjd6tv6g9/processed/1_test',
 '/tmp/tmpjd6tv6g9/processed/2_test',
 '/tmp/tmpjd6tv6g9/processed/3_test',
 '/tmp/tmpjd6tv6g9/processed/4_test']

In [24]:
train_dataset, test_dataset = ExtSumProcessedData().splits(root=save_path)

#### Inspect Data

In [25]:
import torch
bert_format_data = torch.load(train_files[0])
print(len(bert_format_data))
bert_format_data[0].keys()

200


dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [26]:
bert_format_data[0]['labels']

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### [Option 2] Reuse Preprocessed  data from [BERTSUM Repo](https://github.com/nlpyang/BertSum)

In [23]:
# the data path used to downloaded the preprocessed data from BERTSUM Repo.
# if you have downloaded the dataset, change the code to use that path where the dataset is.
PROCESSED_DATA_PATH = "./temp_data5/"# TemporaryDirectory().name
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
#data_path = "./temp_data5/"
#PROCESSED_DATA_PATH = data_path

In [46]:
if USE_PREPROCSSED_DATA:
    download_path = CNNDMBertSumProcessedData.download(local_path=PROCESSED_DATA_PATH)
    train_dataset, test_dataset = ExtSumProcessedData().splits(root=download_path)
    

In [47]:
train_dataset.file_list

['./temp_data5/processed_data/cnndm.train.22.bert.pt',
 './temp_data5/processed_data/cnndm.train.78.bert.pt',
 './temp_data5/processed_data/cnndm.train.88.bert.pt',
 './temp_data5/processed_data/cnndm.train.51.bert.pt',
 './temp_data5/processed_data/cnndm.train.5.bert.pt',
 './temp_data5/processed_data/cnndm.train.120.bert.pt',
 './temp_data5/processed_data/cnndm.train.114.bert.pt',
 './temp_data5/processed_data/cnndm.train.140.bert.pt',
 './temp_data5/processed_data/cnndm.train.87.bert.pt',
 './temp_data5/processed_data/cnndm.train.104.bert.pt',
 './temp_data5/processed_data/cnndm.train.94.bert.pt',
 './temp_data5/processed_data/cnndm.train.59.bert.pt',
 './temp_data5/processed_data/cnndm.train.30.bert.pt',
 './temp_data5/processed_data/cnndm.train.44.bert.pt',
 './temp_data5/processed_data/cnndm.train.73.bert.pt',
 './temp_data5/processed_data/cnndm.train.56.bert.pt',
 './temp_data5/processed_data/cnndm.train.58.bert.pt',
 './temp_data5/processed_data/cnndm.train.83.bert.pt',
 './tem

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [78]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name

# batch size, unit is the number of tokens
BATCH_SIZE = 3000

# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

MAX_POS_LENGTH = 1025

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100

# total number of steps for training
MAX_STEPS=5e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [79]:
summarizer = ExtractiveSummarizer(MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)

In [80]:
summarizer.fit(
            train_dataset,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
        )




Iteration: 0it [00:00, ?it/s]


Iteration: 1it [00:00,  5.33it/s]


Iteration: 2it [00:00,  5.20it/s]


Iteration: 3it [00:00,  5.29it/s]


Iteration: 4it [00:00,  5.24it/s]


Iteration: 5it [00:00,  5.35it/s]


Iteration: 6it [00:01,  5.29it/s]


Iteration: 7it [00:01,  5.37it/s]


Iteration: 8it [00:01,  5.28it/s]


Iteration: 9it [00:01,  5.35it/s]


Iteration: 10it [00:01,  5.29it/s]


Iteration: 11it [00:02,  5.34it/s]


Iteration: 12it [00:02,  5.29it/s]


Iteration: 13it [00:02,  5.41it/s]


Iteration: 14it [00:02,  5.33it/s]


Iteration: 15it [00:02,  5.43it/s]


Iteration: 16it [00:03,  5.32it/s]


Iteration: 17it [00:03,  5.40it/s]


Iteration: 18it [00:03,  5.30it/s]


Iteration: 19it [00:03,  5.39it/s]


Iteration: 20it [00:03,  5.31it/s]


Iteration: 21it [00:03,  5.42it/s]


Iteration: 22it [00:04,  5.18it/s]


Iteration: 23it [00:04,  5.33it/s]


Iteration: 24it [00:04,  5.24it/s]


Iteration: 25it [00:04,  5.35it/s]


Iteration: 26it [00:04,  5.28it/s]


Iteration: 2

Iteration: 176it [00:33,  5.30it/s]


Iteration: 177it [00:33,  5.42it/s]


Iteration: 178it [00:33,  5.32it/s]


Iteration: 179it [00:34,  5.41it/s]


Iteration: 180it [00:34,  5.31it/s]


Iteration: 181it [00:34,  6.07it/s]


Iteration: 182it [00:34,  5.68it/s]


Iteration: 183it [00:34,  5.68it/s]


Iteration: 184it [00:34,  5.49it/s]


Iteration: 185it [00:35,  5.55it/s]


Iteration: 186it [00:35,  5.40it/s]


Iteration: 187it [00:35,  5.49it/s]


Iteration: 188it [00:35,  5.36it/s]


Iteration: 189it [00:35,  5.45it/s]


Iteration: 190it [00:36,  5.32it/s]


Iteration: 191it [00:36,  5.38it/s]


Iteration: 192it [00:36,  5.30it/s]


Iteration: 193it [00:36,  5.41it/s]


Iteration: 194it [00:36,  5.31it/s]


Iteration: 195it [00:36,  5.42it/s]


Iteration: 196it [00:37,  5.32it/s]


Iteration: 197it [00:37,  5.43it/s]


Iteration: 198it [00:37,  5.33it/s]


Iteration: 199it [00:37,  5.41it/s]


Iteration: 200it [00:37,  5.29it/s]


Iteration: 201it [00:38,  5.40it/s]

timestamp: 12/03/2020 03:46:53, average loss: 11.037837, time duration: 37.886213,
                            number of examples in current reporting: 1003, step 100
                            out of total 500





Iteration: 202it [00:38,  5.31it/s]


Iteration: 203it [00:38,  5.42it/s]


Iteration: 204it [00:38,  5.32it/s]


Iteration: 205it [00:38,  5.43it/s]


Iteration: 206it [00:39,  5.33it/s]


Iteration: 207it [00:39,  5.42it/s]


Iteration: 208it [00:39,  5.24it/s]


Iteration: 209it [00:39,  5.36it/s]


Iteration: 210it [00:39,  5.28it/s]


Iteration: 211it [00:39,  5.40it/s]


Iteration: 212it [00:40,  5.28it/s]


Iteration: 213it [00:40,  5.39it/s]


Iteration: 214it [00:40,  5.28it/s]


Iteration: 215it [00:40,  5.37it/s]


Iteration: 216it [00:40,  5.27it/s]


Iteration: 217it [00:41,  5.27it/s]


Iteration: 218it [00:41,  5.14it/s]


Iteration: 219it [00:41,  5.29it/s]


Iteration: 220it [00:41,  5.23it/s]


Iteration: 221it [00:41,  5.35it/s]


Iteration: 222it [00:42,  5.26it/s]


Iteration: 223it [00:42,  5.38it/s]


Iteration: 224it [00:42,  5.27it/s]


Iteration: 225it [00:42,  5.38it/s]


Iteration: 226it [00:42,  5.20it/s]


Iteration: 227it [00:42,  5.34it/s]


Iteration

Iteration: 377it [01:11,  5.39it/s]


Iteration: 378it [01:11,  5.28it/s]


Iteration: 379it [01:11,  5.31it/s]


Iteration: 380it [01:11,  5.24it/s]


Iteration: 381it [01:11,  5.37it/s]


Iteration: 382it [01:12,  5.29it/s]


Iteration: 383it [01:12,  5.40it/s]


Iteration: 384it [01:12,  5.31it/s]


Iteration: 385it [01:12,  5.41it/s]


Iteration: 386it [01:12,  5.31it/s]


Iteration: 387it [01:13,  5.38it/s]


Iteration: 388it [01:13,  5.29it/s]


Iteration: 389it [01:13,  5.40it/s]


Iteration: 390it [01:13,  5.31it/s]


Iteration: 391it [01:13,  5.42it/s]


Iteration: 392it [01:13,  5.32it/s]


Iteration: 393it [01:14,  5.43it/s]


Iteration: 394it [01:14,  5.31it/s]


Iteration: 395it [01:14,  5.44it/s]


Iteration: 396it [01:14,  5.33it/s]


Iteration: 397it [01:14,  5.43it/s]


Iteration: 398it [01:15,  5.32it/s]


Iteration: 399it [01:16,  2.43it/s]


Iteration: 400it [01:16,  2.88it/s]


Iteration: 401it [01:16,  3.63it/s]

...

Iteration: 776it [02:27,  5.29it/s]


Iteration: 777it [02:27,  5.38it/s]


Iteration: 778it [02:27,  5.25it/s]


Iteration: 779it [02:27,  5.36it/s]


Iteration: 780it [02:27,  5.28it/s]


Iteration: 781it [02:27,  5.34it/s]


Iteration: 782it [02:28,  5.24it/s]


Iteration: 783it [02:28,  5.33it/s]


Iteration: 784it [02:28,  5.22it/s]


Iteration: 785it [02:28,  5.33it/s]


Iteration: 786it [02:28,  5.20it/s]


Iteration: 787it [02:29,  5.31it/s]


Iteration: 788it [02:29,  5.20it/s]


Iteration: 789it [02:29,  5.34it/s]


Iteration: 790it [02:29,  5.26it/s]


Iteration: 791it [02:29,  5.38it/s]


Iteration: 792it [02:30,  5.25it/s]


Iteration: 793it [02:30,  5.37it/s]


Iteration: 794it [02:30,  5.23it/s]


Iteration: 795it [02:30,  5.34it/s]


Iteration: 796it [02:31,  2.22it/s]


Iteration: 797it [02:31,  2.71it/s]


Iteration: 798it [02:32,  3.14it/s]


Iteration: 799it [02:32,  3.63it/s]


Iteration: 800it [02:32,  3.96it/s]


Iteration: 801it [02:32,  4.35it/s]

timestamp: 12/03/2020 03:48:48, average loss: 10.414518, time duration: 38.863929,
                            number of examples in current reporting: 1010, step 400
                            out of total 500





Iteration: 802it [02:32,  4.51it/s]


Iteration: 803it [02:33,  4.80it/s]


Iteration: 804it [02:33,  4.84it/s]


Iteration: 805it [02:33,  5.06it/s]


Iteration: 806it [02:33,  5.08it/s]


Iteration: 807it [02:33,  5.19it/s]


Iteration: 808it [02:33,  5.16it/s]


Iteration: 809it [02:34,  5.30it/s]


Iteration: 810it [02:34,  5.24it/s]


Iteration: 811it [02:34,  5.36it/s]


Iteration: 812it [02:34,  5.29it/s]


Iteration: 813it [02:34,  5.38it/s]


Iteration: 814it [02:35,  5.28it/s]


Iteration: 815it [02:35,  5.39it/s]


Iteration: 816it [02:35,  5.29it/s]


Iteration: 817it [02:35,  5.34it/s]


Iteration: 818it [02:35,  5.28it/s]


Iteration: 819it [02:36,  5.39it/s]


Iteration: 820it [02:36,  5.29it/s]


Iteration: 821it [02:36,  5.38it/s]


Iteration: 822it [02:36,  5.28it/s]


Iteration: 823it [02:36,  5.38it/s]


Iteration: 824it [02:36,  5.29it/s]


Iteration: 825it [02:37,  5.41it/s]


Iteration: 826it [02:37,  5.29it/s]


Iteration: 827it [02:37,  5.39it/s]


Iteration

Iteration: 976it [03:05,  5.33it/s]


Iteration: 977it [03:05,  5.44it/s]


Iteration: 978it [03:05,  5.30it/s]


Iteration: 979it [03:05,  5.41it/s]


Iteration: 980it [03:06,  5.30it/s]


Iteration: 981it [03:06,  5.40it/s]


Iteration: 982it [03:06,  5.31it/s]


Iteration: 983it [03:06,  5.43it/s]


Iteration: 984it [03:06,  5.31it/s]


Iteration: 985it [03:07,  5.42it/s]


Iteration: 986it [03:07,  5.34it/s]


Iteration: 987it [03:07,  5.44it/s]


Iteration: 988it [03:07,  5.33it/s]


Iteration: 989it [03:07,  5.43it/s]


Iteration: 990it [03:08,  5.32it/s]


Iteration: 991it [03:08,  5.43it/s]


Iteration: 992it [03:08,  5.31it/s]


Iteration: 993it [03:08,  5.41it/s]


Iteration: 994it [03:08,  5.33it/s]


Iteration: 995it [03:08,  5.44it/s]


Iteration: 996it [03:09,  5.33it/s]


Iteration: 997it [03:09,  5.44it/s]


Iteration: 998it [03:09,  5.35it/s]


Iteration: 999it [03:09,  5.45it/s]


Iteration: 1000it [03:09,  5.34it/s]


Iteration: 1001it [03:10,  5.44it/s]

timestamp: 12/03/2020 03:49:25, average loss: 10.397894, time duration: 37.444429,
                            number of examples in current reporting: 1006, step 500
                            out of total 500


In [49]:
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)

saving through pytorch


In [22]:
# for loading a previous saved model
# import torch
# summarizer.model = torch.load("cnndm_transformersum_distilbert-base-uncased_bertsum_processed_data.pt")

/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'transformers.modeling_distilbert.DistilBertModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [50]:
target = [i['tgt_txt'] for i in test_dataset]

In [51]:
len(target)

11489

In [52]:
test_dataset[0].keys()

dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [68]:
%%time
prediction = summarizer.predict(test_dataset, num_gpus=NUM_GPUS, batch_size=128)

Scoring: 100%|██████████| 90/90 [00:20<00:00,  4.56it/s]


CPU times: user 52.3 s, sys: 22.2 s, total: 1min 14s
Wall time: 22.1 s


In [69]:
len(prediction)

11489

In [55]:
RESULT_DIR = TemporaryDirectory().name

In [81]:
rouge_score = get_rouge(prediction, target, RESULT_DIR) ## 1025

11489
11489


2020-03-12 03:50:09,821 [MainThread  ] [INFO ]  Writing summaries.
INFO:global:Writing summaries.
2020-03-12 03:50:09,823 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpskxkhhwo/tmpsapoy35m/system and model files to /tmp/tmpskxkhhwo/tmpsapoy35m/model.
INFO:global:Processing summaries. Saving system files to /tmp/tmpskxkhhwo/tmpsapoy35m/system and model files to /tmp/tmpskxkhhwo/tmpsapoy35m/model.
2020-03-12 03:50:09,824 [MainThread  ] [INFO ]  Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-50-08/candidate/.
INFO:global:Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-50-08/candidate/.
2020-03-12 03:50:10,877 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmpskxkhhwo/tmpsapoy35m/system.
INFO:global:Saved processed files to /tmp/tmpskxkhhwo/tmpsapoy35m/system.
2020-03-12 03:50:10,878 [MainThread  ] [INFO ]  Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-50-08/reference/.
INFO:global:Processing files in /tmp/tmps

---------------------------------------------
1 ROUGE-1 Average_R: 0.52178 (95%-conf.int. 0.51871 - 0.52495)
1 ROUGE-1 Average_P: 0.34769 (95%-conf.int. 0.34545 - 0.34993)
1 ROUGE-1 Average_F: 0.40331 (95%-conf.int. 0.40117 - 0.40544)
---------------------------------------------
1 ROUGE-2 Average_R: 0.22814 (95%-conf.int. 0.22534 - 0.23102)
1 ROUGE-2 Average_P: 0.15179 (95%-conf.int. 0.14984 - 0.15375)
1 ROUGE-2 Average_F: 0.17599 (95%-conf.int. 0.17394 - 0.17813)
---------------------------------------------
1 ROUGE-L Average_R: 0.47364 (95%-conf.int. 0.47064 - 0.47654)
1 ROUGE-L Average_P: 0.31615 (95%-conf.int. 0.31388 - 0.31837)
1 ROUGE-L Average_F: 0.36646 (95%-conf.int. 0.36435 - 0.36866)



In [70]:
rouge_score = get_rouge(prediction, target, RESULT_DIR) ## 1024

11489
11489


2020-03-12 03:32:26,907 [MainThread  ] [INFO ]  Writing summaries.
INFO:global:Writing summaries.
2020-03-12 03:32:26,908 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpskxkhhwo/tmpzcxohf9d/system and model files to /tmp/tmpskxkhhwo/tmpzcxohf9d/model.
INFO:global:Processing summaries. Saving system files to /tmp/tmpskxkhhwo/tmpzcxohf9d/system and model files to /tmp/tmpskxkhhwo/tmpzcxohf9d/model.
2020-03-12 03:32:26,909 [MainThread  ] [INFO ]  Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-32-25/candidate/.
INFO:global:Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-32-25/candidate/.
2020-03-12 03:32:28,012 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmpskxkhhwo/tmpzcxohf9d/system.
INFO:global:Saved processed files to /tmp/tmpskxkhhwo/tmpzcxohf9d/system.
2020-03-12 03:32:28,014 [MainThread  ] [INFO ]  Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-32-25/reference/.
INFO:global:Processing files in /tmp/tmps

---------------------------------------------
1 ROUGE-1 Average_R: 0.52178 (95%-conf.int. 0.51871 - 0.52495)
1 ROUGE-1 Average_P: 0.34769 (95%-conf.int. 0.34545 - 0.34993)
1 ROUGE-1 Average_F: 0.40331 (95%-conf.int. 0.40117 - 0.40544)
---------------------------------------------
1 ROUGE-2 Average_R: 0.22814 (95%-conf.int. 0.22534 - 0.23102)
1 ROUGE-2 Average_P: 0.15179 (95%-conf.int. 0.14984 - 0.15375)
1 ROUGE-2 Average_F: 0.17599 (95%-conf.int. 0.17394 - 0.17813)
---------------------------------------------
1 ROUGE-L Average_R: 0.47364 (95%-conf.int. 0.47064 - 0.47654)
1 ROUGE-L Average_P: 0.31615 (95%-conf.int. 0.31388 - 0.31837)
1 ROUGE-L Average_F: 0.36646 (95%-conf.int. 0.36435 - 0.36866)



In [56]:
rouge_score = get_rouge(prediction, target, RESULT_DIR) ## 768

11489
11489


2020-03-12 03:10:18,533 [MainThread  ] [INFO ]  Writing summaries.
INFO:global:Writing summaries.
2020-03-12 03:10:18,535 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpskxkhhwo/tmp8m2ho5lo/system and model files to /tmp/tmpskxkhhwo/tmp8m2ho5lo/model.
INFO:global:Processing summaries. Saving system files to /tmp/tmpskxkhhwo/tmp8m2ho5lo/system and model files to /tmp/tmpskxkhhwo/tmp8m2ho5lo/model.
2020-03-12 03:10:18,536 [MainThread  ] [INFO ]  Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-10-17/candidate/.
INFO:global:Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-10-17/candidate/.
2020-03-12 03:10:22,823 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmpskxkhhwo/tmp8m2ho5lo/system.
INFO:global:Saved processed files to /tmp/tmpskxkhhwo/tmp8m2ho5lo/system.
2020-03-12 03:10:22,824 [MainThread  ] [INFO ]  Processing files in /tmp/tmpskxkhhwo/rouge-tmp-2020-03-12-03-10-17/reference/.
INFO:global:Processing files in /tmp/tmps

---------------------------------------------
1 ROUGE-1 Average_R: 0.50706 (95%-conf.int. 0.50393 - 0.50999)
1 ROUGE-1 Average_P: 0.34773 (95%-conf.int. 0.34536 - 0.35005)
1 ROUGE-1 Average_F: 0.39874 (95%-conf.int. 0.39652 - 0.40106)
---------------------------------------------
1 ROUGE-2 Average_R: 0.21946 (95%-conf.int. 0.21676 - 0.22224)
1 ROUGE-2 Average_P: 0.15089 (95%-conf.int. 0.14884 - 0.15292)
1 ROUGE-2 Average_F: 0.17269 (95%-conf.int. 0.17064 - 0.17486)
---------------------------------------------
1 ROUGE-L Average_R: 0.46072 (95%-conf.int. 0.45788 - 0.46352)
1 ROUGE-L Average_P: 0.31652 (95%-conf.int. 0.31422 - 0.31880)
1 ROUGE-L Average_F: 0.36268 (95%-conf.int. 0.36051 - 0.36500)



In [57]:
test_dataset[0]['tgt_txt']

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'

In [58]:
prediction[0]

'he was flown back to chicago via air ambulance on march 20 , but he died on sunday .<q>andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .<q>a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .'

In [60]:
len(test_dataset[0]['src_txt'])

17

In [64]:
sum([len(o.split()) for o in test_dataset[20]['src_txt']])

630

In [44]:
# for testing
sb.glue("rouge_2_f_score", rouge_score['rouge_2_f_score'])

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if os.path.exists(RESULT_DIR):
    shutil.rmtree(RESULT_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)